## Task Development




In [ ]:
# objective: migrate the protocol to the new structure
# protocol is currently split between two parts: milling and options
# milling is a dict of dicts for milling stages. These have "magic" keys that assign them to specific tasks
# options are just a set of options that don't really have a specific organisation
# supervise: is a flag set for each task, 

# the advantage is that the parameters for each task are all in one place
# the protocol also contains 'complete' information about the tasks, 
# previously only the milling stages were in the protocol of the lamella. "options" were globally set
# the new structure allows full individualisation of each task for each lamella.

# moreover, the workflow/task structure does not rely on fixed methods, so indiviual tasks can be added/removed/altered
# without having to change the protocol structure.

# the downside is that the state-machine is not as clear, and we need to update the logic for whether we can proceed to each task
# also, we need to update the logic to detemrine what position the stage shold be at for each task
# previously this was handled by explicitly definign the method such that we knew what the stage position was

# what to do about 'global' parameters, such as ml models.

# WORKFLOW
#     TASK-01
#     TASK-02
#     TASK-03


# PROTOCOL
#    TASK
#       name: TASK-01
#       description: TASK-01
#       supervise: bool
#       parameters: Dict
#       milling: Dict[str, List[MillingStage]]


# EXAMPLE:
# - name: Rough Milling
#   description: Rough milling
#   parameters:
#       supervise: true
#   milling:
#   - name: Microexpansion
#     milling: ...
#   - name: Rough Milling 01
#     milling: ...
#   - name: Rough Milling 02
#     milling: ...

# - name: Spot Burn Fiducials
#   description: Spot burn fiducials
#   parameters:
#       supervise: true
#       milling_current: 

In [ ]:
from dataclasses import dataclass

from typing import List, Dict
from fibsem.milling import FibsemMillingStage, get_milling_stages


@dataclass
class Task:
    name: str
    description: str
    parameters: dict
    milling: List[FibsemMillingStage]
    supervise: bool = False

    @classmethod
    def from_dict(cls, d: dict):
        name = d["name"]
        description = d["description"]
        parameters = d["parameters"]
        if "milling" in d:
            milling = get_milling_stages("milling", d)
        else:
            milling = []
        supervise = d.get("supervise", True)
        return cls(name=name,
                   description=description,
                   parameters=parameters,
                   milling=milling,
                   supervise=supervise)
    
    def to_dict(self) -> Dict:
        d = {
            "name": self.name,
            "description": self.description,
            "parameters": self.parameters,
            "milling": [stage.to_dict() for stage in self.milling],
            "supervise": self.supervise
        }
        return d
    
@dataclass
class OpenFIBSEMProtocol:
    name: str
    description: str
    tasks: List[Task]
    grid_tasks: List[Task] = None
    configuration: dict = None

    def __post_init__(self):
        if self.configuration is None:
            self.configuration = {}

    @classmethod
    def from_dict(cls, d: dict):
        name = d["name"]
        description = d["description"]
        tasks = [Task.from_dict(task) for task in d["tasks"]]
        if "grid-tasks" in d:
            grid_tasks = [Task.from_dict(task) for task in d["grid-tasks"]]
        else:
            grid_tasks = []
        return cls(name=name,
                   description=description,
                   tasks=tasks,
                    grid_tasks=grid_tasks,
                   configuration=d.get("configuration", {}))
    
    def to_dict(self) -> Dict:
        d = {
            "name": self.name,
            "description": self.description,
            "tasks": [task.to_dict() for task in self.tasks],
            "grid-tasks": [task.to_dict() for task in self.grid_tasks],
            "configuration": self.configuration
        }
        return d


import yaml
PROTOCOL_PATH = "/home/patrick/github/autolamella/autolamella/protocol/dev-protocol-tasks.yaml"


with open(PROTOCOL_PATH, "r") as f:
    ddict = yaml.safe_load(f)
protocol = OpenFIBSEMProtocol.from_dict(ddict)
print(protocol.name)
print(protocol.description)

for task in protocol.tasks:
    print(task.name + " " + task.description)
    pprint(task.parameters)


for task in protocol.grid_tasks:
    print(task.name + " " + task.description)
    pprint(task.parameters)

